In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"

# Download and convert data

In [3]:
from tqdm import tqdm

from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysis
from document_segmentation.settings import RENATE_ANALYSIS_DIR

N = None

RENATE_ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)

sheet = RenateAnalysis()


existing_docs = {
    path.stem for path in RENATE_ANALYSIS_DIR.glob("Globdoc_*.json") if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),
    desc="Writing documents",
    unit="doc",
):
    document_file = RENATE_ANALYSIS_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents: 0doc [00:00, ?doc/s]


In [4]:
import logging

from tqdm import tqdm

from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysisInv
from document_segmentation.settings import RENATE_ANALYSIS_DIR, RENATE_ANALYSIS_SHEETS

N = None


sheet = RenateAnalysisInv(RENATE_ANALYSIS_SHEETS[0])  # TODO: use both sheets

for document in tqdm(
    sheet.to_documents(n=N), desc="Writing documents", unit="doc", total=26
):
    document_file = RENATE_ANALYSIS_DIR / f"{document.id}.json"

    if document_file.exists():
        logging.info(f"Document {document.id} already exists, skipping")
    else:
        with document_file.open("xt") as f:
            f.write(document.model_dump_json())
            f.write("\n")

Writing documents: 100%|██████████| 26/26 [00:11<00:00,  2.33doc/s]


# Load Data

In [5]:
%autoreload now

In [6]:
TRAINING_DATA = 0.8

In [7]:
from document_segmentation.model.dataset import DocumentDataset

dataset: DocumentDataset = DocumentDataset.from_dir(RENATE_ANALYSIS_DIR)
dataset.shuffle()

len(dataset)

Reading JSON files: 100%|██████████| 104/104 [00:00<00:00, 160.10file/s]


104

In [8]:
dataset._class_counts()

Counter({<Label.IN: 1>: 1907,
         <Label.BEGIN: 0>: 104,
         <Label.END: 2>: 100,
         <Label.OUT: 3>: 73})

In [9]:
dataset.class_weights()

[0.9904761904761905,
 0.05450733752620545,
 1.0297029702970297,
 1.4054054054054055]

In [10]:
training_data, test_data = dataset.split(TRAINING_DATA)

In [11]:
training_data._class_counts()

Counter({<Label.IN: 1>: 1613,
         <Label.BEGIN: 0>: 83,
         <Label.END: 2>: 79,
         <Label.OUT: 3>: 54})

In [12]:
test_data._class_counts()

Counter({<Label.IN: 1>: 294,
         <Label.BEGIN: 0>: 21,
         <Label.END: 2>: 21,
         <Label.OUT: 3>: 19})

# Train Model

In [13]:
import torch

BATCH_SIZE = 64
EPOCHS = 10
WEIGHTS = dataset.class_weights()

In [14]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [15]:
tagger._device

'mps'

In [16]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_features=4, bias=True)
  (_soft

In [17]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS.to(tagger._device))

Training: 100%|██████████| 93/93 [01:46<00:00,  1.15s/batch]


[Loss:	0.124]


Training: 100%|██████████| 93/93 [00:05<00:00, 17.93batch/s]


[Loss:	0.123]


Training: 100%|██████████| 93/93 [00:05<00:00, 17.81batch/s]


...

[Loss:	0.074]


Training: 100%|██████████| 93/93 [00:05<00:00, 18.06batch/s]


[Loss:	0.074]


Training: 100%|██████████| 93/93 [00:05<00:00, 18.10batch/s]

[Loss:	0.074]


In [18]:
with open("page_sequence_tagger.pt", "xb") as f:
    torch.save(tagger, f)

# Evaluate Model

In [19]:
import sys

precision, recall, f1, accuracy = tagger.eval_(test_data, BATCH_SIZE, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating:   0%|          | 0/23 [00:00<?, ?batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_8737_0089.jpg	Van Bengale, onder dato 28„' 8ber: A. o 1705.; voo	[0.9998795986175537, 4.716290277428925e-05, 5.450770186143927e-05, 1.8789738533087075e-05]


Evaluating:   4%|▍         | 1/23 [00:00<00:04,  5.00batch/s]

END	END	NL-HaNA_1.04.02_8737_0090.jpg	Van Bengale onder dato 28:' xber: A:o 1705.; waar 	[0.0003245954285375774, 0.00012509663065429777, 0.9995437264442444, 6.612697234231746e-06]


...

BEGIN	IN	NL-HaNA_1.04.02_1547_0553.jpg	. . . . . . . . soldaet. . . . . . . . . . . . . 1	[0.9951327443122864, 0.0022832206450402737, 0.000586451671551913, 0.0019975262694060802]
IN	IN	NL-HaNA_1.04.02_1547_0554.jpg	807. Persoonen P„r Transport; Naemen Toenaem en ge	[0.20820622146129608, 0.682439386844635, 0.10669571906328201, 0.0026586188469082117]
IN	IN	NL-HaNA_1.04.02_1547_0555.jpg	te staats presente qualiteijt, en winnende 7 't sc	[0.07234015315771103, 0.7741838693618774, 0.1525031477212906, 0.0009728281875140965]
IN	IN	NL-HaNA_1.04.02_1547_0556.jpg	838. Persoonen P„r Transport; Naemen Toenaemen geb	[0.05923589691519737, 0.7543302774429321, 0.18561431765556335, 0.0008195725968107581]
IN	IN	NL-HaNA_1.04.02_1547_0557.jpg	Plaatsd presente qualiteijt, en winnenden 't schip	[0.05670130252838135, 0.7429326176643372, 0.19957104325294495, 0.0007950470899231732]
IN	IN	NL-HaNA_1.04.02_1547_0558.jpg	Naemen Toenaem en geboorte Plaats; 861. Persoonen 	[0.05579344928264618, 0.7378417253494263, 0.

Evaluating:  96%|█████████▌| 22/23 [00:24<00:02,  2.41s/batch]

BEGIN	BEGIN	NL-HaNA_1.04.02_1081_0045.jpg	Saterdach den 22 octob 1622; Alsoo de scheepen gro	[0.9946178793907166, 0.0024721540976315737, 0.0006270447629503906, 0.0022829489316791296]
IN	IN	NL-HaNA_1.04.02_1081_0046.jpg	Mede is goedt geuonden dat Sr. Cornelis van Neyenr	[0.21198444068431854, 0.6825459599494934, 0.10268929600715637, 0.0027802332770079374]
IN	IN	NL-HaNA_1.04.02_1081_0047.jpg	Sondach den 6 novemb. 1622„; -; Alsoo goedt geuond	[0.07224947959184647, 0.7775067090988159, 0.14925150573253632, 0.0009923357283696532]
IN	IN	NL-HaNA_1.04.02_1081_0048.jpg	Achter de Eijlanden aende zuijtzijde vande baeij; 	[0.05883494019508362, 0.7577874064445496, 0.1825484335422516, 0.0008292103884741664]
IN	IN	NL-HaNA_1.04.02_1081_0049.jpg	Herwaerts spoeije om onse Instructie ende goet voo	[0.05623100325465202, 0.7462183237075806, 0.19674964249134064, 0.0008010139572434127]
IN	IN	NL-HaNA_1.04.02_1081_0050.jpg	Saterdach den 19 Novemb 1622; Alsoo op gisteren go	[0.05531591176986694, 0.740911662578582

Evaluating: 100%|██████████| 23/23 [00:25<00:00,  1.09s/batch]

BEGIN	BEGIN	NL-HaNA_1.04.02_1647_0632.jpg	Van Macassar Anno 1701—; Vervolgens quam by den He	[0.9998655319213867, 6.913366087246686e-05, 4.472258660825901e-05, 2.0562354620778933e-05]
IN	IN	NL-HaNA_1.04.02_1647_0633.jpg	Van Macassar Anno 1701; Van Macassar Anno 1701.; a	[0.0004996329662390053, 0.9992245435714722, 0.0002733964647632092, 2.3458528630726505e-06]
IN	IN	NL-HaNA_1.04.02_1647_0634.jpg	Van Macassar Anno 1701; Van Macassar Anno 1701; va	[0.00025200346135534346, 0.9994682669639587, 0.00027840209077112377, 1.3643231113746879e-06]
IN	IN	NL-HaNA_1.04.02_1647_0635.jpg	Van Macassar Anno 1701; Van Macassar Anno 1701; ve	[0.00023022093228064477, 0.9994932413101196, 0.0002751544816419482, 1.252047582056548e-06]
IN	IN	NL-HaNA_1.04.02_1647_0636.jpg	Van Macassar Anno 1701; Van Macassar A„o 1701; het	[0.00023508621961809695, 0.9994787573814392, 0.0002849498705472797, 1.2003226856904803e-06]
IN	IN	NL-HaNA_1.04.02_1647_0637.jpg	Van Macassar Anno 1701; Van Macassar A„o 1701.; mo	[0.00023212775

In [20]:
import csv

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{metric.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

Metric	BEGIN	IN	END	OUT
MulticlassPrecision	0.8261	0.9863	0.9130	0.9375
MulticlassRecall	0.9048	0.9830	1.0000	0.7895
MulticlassF1Score	0.8636	0.9847	0.9545	0.8571
MulticlassF1Score (micro average):	0.9690
